<a href="https://colab.research.google.com/github/lucianomcsilva/ANUNCIOS_OLX/blob/main/%5BRIWRS%5D_6_An%C3%A1lise_de_Sentimentos_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregar as bibliotecas e dados

## Bibliotecas

### Instalações via PIP

In [2]:
!pip install unidecode
!pip install fractions
!pip install googletrans
!pip install ibm_watson
!pip install ibm_cloud_sdk_core
!pip install dotenv


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement fractions (from versions: none)
ERROR: No matching distribution found for fractions
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Using cached dotenv-0.0.4.tar.gz (2.0 kB)
  Using cached dotenv-0.0.2.tar.gz (6.7 kB)
  Using cached dotenv-0.0.1.tar.gz (6.5 kB)
ERROR: Could not find a version that satisfies the requirement dotenv (from version

### Importações

In [3]:
# Load EDA Pkgs
import pandas as pd
import numpy as np
import random

# Load ML Pkgs
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('rslp')
from textblob import TextBlob

#charts & others stuff
from unidecode import unidecode
import matplotlib.pyplot as plt
import re
import string
from fractions import Fraction
import json
import os
from googletrans import Translator
translator = Translator()


# from dotenv import load_dotenv
# load_dotenv()
# load_dotenv('../ibm-credentials.env')

# IBM Watson
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson import LanguageTranslatorV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1  import Features, CategoriesOptions, SentimentOptions



# NATURAL_LANGUAGE_UNDERSTANDING_APIKEY = os.environ['NATURAL_LANGUAGE_UNDERSTANDING_APIKEY']
# NATURAL_LANGUAGE_UNDERSTANDING_URL    = os.environ['NATURAL_LANGUAGE_UNDERSTANDING_URL']

# LANGUAGE_TRANSLATOR_APIKEY = os.environ['LANGUAGE_TRANSLATOR_APIKEY']
# LANGUAGE_TRANSLATOR_URL    = os.environ['LANGUAGE_TRANSLATOR_URL']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


## Carrega o Arquivo

In [5]:
def RoundSentiment(x):
  if x > 0.5:
    return 1
  elif x < -0.5:
    return -1
  return 0


df_o = pd.read_csv(f"https://raw.githubusercontent.com/lucianomcsilva/RIWRS/main/tweets_com_ibm_watson_e_ingles.csv")
df_o.drop(df_o.columns[df_o.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
#df[['id_str', 'text', 'screen_name', 'search_term', 'search_term_group', 'source', 'ibm_watson_sentiment', 'ibm_watson_sentiment_label']].sample(10)
df = df_o[['id_str', 'text', 'screen_name', 'search_term', 'search_term_group', 'source', 'ibm_watson_sentiment']].loc[df_o['search_term_group'].ne('star+')]
df['sentiment'] = df['ibm_watson_sentiment'].apply(lambda x: RoundSentiment(x))

# Naive Bayes

## Prepara uma amostra com 100 registros para validarmos a acuracia por nos mesmos

In [ ]:
# Faz uma amostra apenas com 200 registros aleatorios
# Iremos descatar alguns que classificarmos como neutro
df_nb = df.loc[df['sentiment'].ne(0)].sample(200)

#Salva o arquivo para podermos analisar na mao
df_nb.to_csv("amostra.csv")

## Carrega o Arquivo Revisado

In [ ]:
df_nbr = pd.read_csv('amostras-revisado.csv', delimiter=';')
df_nbr = df_nbr.loc[df_nbr['sentiment_revised'].ne(0)]
#df_nbr_pos = df_nbr.loc[df_nbr['sentiment_revised'].eq(1)]


In [ ]:
df_nbr

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

vectorizer = CountVectorizer()

# Vectorize the text
X = vectorizer.fit_transform(df_nbr['text'])
Y = df_nbr['sentiment_revised']

# Split dataset
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.33,random_state=42)

vocabulary = vectorizer.get_feature_names()
dfX = pd.DataFrame(data=X.toarray(), columns=vocabulary) #.iloc[:,0::2]

In [ ]:
# Building Our Model
clf = MultinomialNB()
clf.fit(x_train,y_train)

print("Accuracy of Model :",clf.score(x_test,y_test))

In [ ]:
clf.get_params()

In [ ]:
test_tweet = ['adorei! recomendo titans.', 
              'Af pessimo',
              'netflix'
             ]
vect = vectorizer.transform(test_tweet).toarray()
vect.shape
clf.predict(vect)

In [ ]:
clf.mo

In [ ]:
# class_prob_sorted = clf.feature_log_prob_[0, :].argsort()[::-1]
# print(np.take(vectorizer.get_feature_names(), class_prob_sorted[:10]))

def get_salient_words(nb_clf, vect, class_ind):
    """Return salient words for given class
    Parameters
    ----------
    nb_clf : a Naive Bayes classifier (e.g. MultinomialNB, BernoulliNB)
    vect : CountVectorizer
    class_ind : int
    Returns
    -------
    list
        a sorted list of (word, log prob) sorted by log probability in descending order.
    """

    words = vect.get_feature_names()
    zipped = list(zip(words, nb_clf.feature_log_prob_[class_ind]))
    sorted_zip = sorted(zipped, key=lambda t: t[1], reverse=True)

    return sorted_zip

neg_salient_top_20 = get_salient_words(clf, vectorizer, 0)[:20]
pos_salient_top_20 = get_salient_words(clf, vectorizer, 1)[:20]



In [ ]:
neg_salient_top_20

In [ ]:
pos_salient_top_20

# NLTK

## Simple case

In [ ]:
def gender_features(word):
  return {'last_letter': word[-2:]}

In [ ]:
from nltk.corpus import names
nltk.download('names')
labeled_names = ([(name, 'male') for name in names.words('male.txt')] + [(name, 'female') for name in names.words('female.txt')])
#import random
#random.shuffle(labeled_names)

In [ ]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)

In [ ]:
classifier.classify(gender_features('Andre'))



## Classifier


In [ ]:
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

In [ ]:
documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]
documents              